## Assignment
Yeah ! Let's start with our actual project. In this assignment we will load the database and do preprocessing tasks.
Ensure you have following packages installed
1. numpy
2. pandas  
( Hope you are familiar with above two modules well )
3. nltk (don't worry, we just need this to remove stopwords while preprocessing)
4. tensorflow
5. keras
6. scikit-learn

In [5]:
'''
Importing essential libraries and functions
'''
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import io
import json

## 1. Loading the IMDB Reviews Dataset

In [6]:
# TODO:
'''
Using pandas load the imdb reviews csv file
Analyse its shape and columns
Check for missing data and remove those rows
'''

# Loading the IMDB Reviews Dataset
dataset_path = 'IMDB_dataset.csv'

df = pd.read_csv(dataset_path)

print("Dataset Shape:", df.shape)
print("Columns:", df.columns)

# checking for missing data
print("Missing data:")
print(df.isnull().sum())

# removing such rows
df.dropna(inplace=True)

#checking 
print("After removing missing data:")
print("Dataset Shape:", df.shape)

df.reset_index(drop=True, inplace=True)

Dataset Shape: (50000, 2)
Columns: Index(['review', 'sentiment'], dtype='object')
Missing data:
review       0
sentiment    0
dtype: int64
After removing missing data:
Dataset Shape: (50000, 2)


## 2. Data Preprocessing

In [9]:
#TODO : Complete the function to preprocess the text data

def preprocessing(sentence):
    # First make the sentence lowercase
    sentence = sentence.lower()
    # Remove all html tags from the sentence i.e replace anything between <> with space
    # Hint use Regular Expression i.e. re.sub()
    sentence = re.sub(r'<.*?>', ' ', sentence)
    
    # Remove all special characters i.e. anything other than alphabets and numbers. Replace them with space
    sentence = re.sub(r'[^a-zA-Z0-9\s]', ' ', sentence)
    
    # Remove all single characters i.e. a-z and A-Z and Replace them with space
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    
    # Remove all multiple spaces and replace them with single space
    sentence = re.sub(r'\s+', ' ', sentence)
    
    # Use the nltk library to remove all stopwords from the sentence
    # stopwords are the words like and, the, is, are etc.
    words = nltk.word_tokenize(sentence)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    sentence = ' '.join(words)
    
    # return the sentence
    return sentence
    
    pass # remove this line after writing the code

In [10]:
# TODO :
nltk.download('stopwords')
nltk.download('punkt')
# Call the preprocessing function for each review in the dataframe and
# save the results in a new list of preprocessed_reviews
preprocessed_reviews = [preprocessing(review) for review in df['review']]
# This list will be your input to the neural network
# We will call this list as X from now on
X = preprocessed_reviews
print(X[:5])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['one reviewers mentioned watching 1 oz episode hooked right exactly happened first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home many aryans muslims gangstas latinos christians italians irish scuffles death stares dodgy dealings shady agreements never far away would say main appeal show due fact goes shows dare forget pretty pictures painted mainstream audiences forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards sold nickel inmates kill order get away well mannered middle class inmates turned prison bitches due lack street sk

In [11]:
# TODO :
# Convert sentiment column in the dataframe to numbers
# Convert positive to 1 and negative to 0 and store it in numpy array
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
# We will call this numpy array as y from now on
y = np.array(df['sentiment'])
print(y[:5])

[1 1 1 0 1]


In [12]:
# TODO : Split the data into training and testing (80-20 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# The train set will be used to train our deep learning models 
# while test set will be used to evaluate how well our model performs 
print("Training data shape:", len(X_train), len(y_train))
print("Testing data shape:", len(X_test), len(y_test))

Training data shape: 40000 40000
Testing data shape: 10000 10000


## Preparing embedding layer
Let's now write the script for our embedding layer. Embedding layer converts our textual data into numeric form. It is then **used as the first layer for the deep learning model like LSTM**.  
To know more about word embedding you may refer to following video
https://www.youtube.com/watch?v=9S0-OC4LFNo  
#### Tokenize the data

In [13]:
# Initialize the tokenizer
word_tokenizer = Tokenizer()
# TODO: Fit the tokenizer on the training data (X_train)
word_tokenizer.fit_on_texts(X_train)
# TODO: Convert training data to sequences of integers
# Hint: Use texts_to_sequences method
X_train_sequences = word_tokenizer.texts_to_sequences(X_train)
# TODO: Convert test data to sequences of integers
# Hint: Use texts_to_sequences method
X_test_sequences = word_tokenizer.texts_to_sequences(X_test)
# End TODO
# Saving the tokenizer in a json file (Already done for you)
# This will be used later for prediction on data in next assignments
tokenizer_json = word_tokenizer.to_json()
with io.open('b3_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
    
# Vocab_length is the number of unique words in our dataset
# Adding 1 to store dimensions for words for which no pretrained word embeddings exist
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

92321

#### Padding

In [14]:
# Padding all reviews to be of same length 'maxlen' words
maxlen = 100
# You can try different dimensions like 50, 100, 200 and 300
# and see how the model performs in next week

# TODO: Pad the training data sequences
# Hint: Use pad_sequences with 'post' padding and maxlen=maxlen
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_padded = pad_sequences(X_train_sequences, padding='post', maxlen=maxlen)

# TODO: Pad the test data sequences
# Hint: Use pad_sequences with 'post' padding and maxlen=maxlen
X_test_padded = pad_sequences(X_test_sequences, padding='post', maxlen=maxlen)
print("Padded training data shape:", X_train_padded.shape)
print("Padded test data shape:", X_test_padded.shape)

Padded training data shape: (40000, 100)
Padded test data shape: (10000, 100)


#### Glove Embedding

In [16]:
# Initialize an empty dictionary for embeddings
embeddings_dictionary = dict()

# Open the GloVe file (a2_glove.6B.100d.txt) with utf-8 encoding
glove_file = open('a2_glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

# TODO : Create an embedding matrix where each row corresponds to the index of the
# unique word in the dataset and each column corresponds to the word vector
# in the GloVe embedding 
# So the matrix will have vocab_length rows and maxlen columns
embedding_dim = 100
embedding_matrix = np.zeros((vocab_length, embedding_dim))
for word, index in word_tokenizer.word_index.items():
    if index < vocab_length:
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (92321, 100)
